# Import the libraries 

In [1]:
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# For training XGBRegressor model
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Load the data

Next, we'll load the training and test data.  

We set `index_col=0` in the code cell below to use the `id` column to index the DataFrame.

In [2]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv", index_col=0)
test = pd.read_csv("../input/30-days-of-ml/test.csv", index_col=0)

# Preview the data
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
2,B,B,A,A,B,D,A,F,A,O,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
3,A,A,A,C,B,D,A,D,A,F,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
4,B,B,A,C,B,D,A,E,C,K,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
6,A,A,A,C,B,D,A,E,A,N,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


The next code cell separates the target (which we assign to `y`) from the training features (which we assign to `features`).

In [3]:
# Separate target from features
y = train['target']
features = train.drop(['target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
2,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
3,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
4,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
6,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Prepare the data

Next, we'll need to handle the categorical columns (`cat0`, `cat1`, ... `cat9`).  

we'll use ordinal encoding and save our encoded features as new variables `X` and `X_test`.

In [4]:
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

X.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
2,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
3,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
4,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
6,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


Next, we break off a validation set from the training data.

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,train_size=0.8,test_size=0.2, random_state=0)

#  Train a model

Now that the data is prepared, the next step is to train a model.  

In the code cell below, we fit a XGBRegressor model to the data.

In [6]:
# Define the model 

model = XGBRegressor(n_estimators=500, learning_rate=0.05)
model.fit(X_train,y_train,early_stopping_rounds=5,eval_set=[(X_valid, y_valid)])
preds_valid=model.predict(X_valid)
print(mean_squared_error(y_valid, preds_valid, squared=False))

[0]	validation_0-rmse:7.38977
[1]	validation_0-rmse:7.02395
[2]	validation_0-rmse:6.67663
[3]	validation_0-rmse:6.34690
[4]	validation_0-rmse:6.03385
[5]	validation_0-rmse:5.73670
[6]	validation_0-rmse:5.45464
[7]	validation_0-rmse:5.18694
[8]	validation_0-rmse:4.93291
[9]	validation_0-rmse:4.69183
[10]	validation_0-rmse:4.46310
[11]	validation_0-rmse:4.24611
[12]	validation_0-rmse:4.04030
[13]	validation_0-rmse:3.84511
[14]	validation_0-rmse:3.66004
[15]	validation_0-rmse:3.48455
[16]	validation_0-rmse:3.31822
[17]	validation_0-rmse:3.16060
[18]	validation_0-rmse:3.01127
[19]	validation_0-rmse:2.86987
[20]	validation_0-rmse:2.73597
[21]	validation_0-rmse:2.60925
[22]	validation_0-rmse:2.48937
[23]	validation_0-rmse:2.37598
[24]	validation_0-rmse:2.26881
[25]	validation_0-rmse:2.16749
[26]	validation_0-rmse:2.07181
[27]	validation_0-rmse:1.98151
[28]	validation_0-rmse:1.89632
[29]	validation_0-rmse:1.81598
[30]	validation_0-rmse:1.74033
[31]	validation_0-rmse:1.66911
[32]	validation_0-

/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


0.722922237489933


In the code cell above, we set `squared=False` to get the root mean squared error (RMSE) on the validation data.

# Generate predictions

We'll begin by using the trained model to generate predictions, which we'll save to a CSV file.

In [ ]:
# Use the model to generate predictions
predictions = model.predict(X_test)
# Save the predictions to a CSV file
output = pd.DataFrame({'Id': X_test.index,'target': predictions})
output.to_csv('predictions_of_insurance.csv', index=False)